In [12]:
import numpy as np

class SlidingPuzzle():
    def __init__(self, num_blocos):
        self.num_blocos = num_blocos

    def _encontra_posicao(self, estado, elemento):
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if estado[i, j] == elemento:
                    return i, j
        return None, None

    def verifica_estados(self, atual, objetivo):
        flag = True
        for i in range(self.num_blocos):
            if flag == False:
                break

            for j in range(self.num_blocos):
                if atual[i, j] != objetivo[i, j]:
                    flag = False;
                    break;

        return flag

    def expande_estado(self, atual):
        novos_estados = []
        linha, coluna = self._encontra_posicao(atual, 0)

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        return novos_estados

    def heuristic(self, atual, objetivo):
        total_dist = 0

        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                elemento = atual[i, j]
                linha, coluna = self._encontra_posicao(objetivo, elemento)

                if  (linha != i or j != coluna):
                    total_dist += 1

        return total_dist



### Busca Gulosa

In [13]:
import heapq

class BuscaGulosa():
    def __init__(self, problema):
        self.problema = problema

    def _verifica_visitado(self, estado, estados_visitados):
        for i in estados_visitados:
            if self.problema.verifica_estados(i, estado):
                return True
        return False

    def busca(self, inicio, fim):
        p_fila = []
        
        # H, ID, elemento
        heapq.heappush(p_fila, (0, 0, inicio))

        id_estado = 0
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0

        while not len(p_fila) == 0:
            atual = heapq.heappop(p_fila)[2]
            estados_visitados.append(atual)

            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                print("Visitando #", cont_estados)
                novos_estados = self.problema.expande_estado(atual)

                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados):
                        id_estado += 1
                        prioridade = self.problema.heuristic(i, fim)

                        print(i, "\nh = ", prioridade)
                        heapq.heappush(p_fila, (prioridade, id_estado, i))

        return solucao_encontrada, estados_visitados, cont_estados


### Chamando a Busca

In [14]:
problema = SlidingPuzzle(3)

start = np.matrix([[8, 2, 3], [1, 6, 4], [7, 0, 5]])
print(start, "\n********\n")

target = np.matrix([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
print(target, "\n********\n")

gulosa = BuscaGulosa(problema)
solucao, visitados, steps = gulosa.busca(start, target)

if solucao:
    print(f"Solução encontrada em {steps}")
else:
    print("Solução não encontrada")

[[8 2 3]
 [1 6 4]
 [7 0 5]] 
********

[[1 2 3]
 [4 5 6]
 [7 8 0]] 
********

Visitando # 1
[[8 2 3]
 [1 0 4]
 [7 6 5]] 
h =  6
[[8 2 3]
 [1 6 4]
 [0 7 5]] 
h =  7
[[8 2 3]
 [1 6 4]
 [7 5 0]] 
h =  5
Visitando # 2
[[8 2 3]
 [1 6 0]
 [7 5 4]] 
h =  6
Visitando # 3
[[8 0 3]
 [1 2 4]
 [7 6 5]] 
h =  7
[[8 2 3]
 [0 1 4]
 [7 6 5]] 
h =  6
[[8 2 3]
 [1 4 0]
 [7 6 5]] 
h =  6
Visitando # 4
[[8 2 0]
 [1 6 3]
 [7 5 4]] 
h =  7
[[8 2 3]
 [1 0 6]
 [7 5 4]] 
h =  5
Visitando # 5
[[8 0 3]
 [1 2 6]
 [7 5 4]] 
h =  6
[[8 2 3]
 [1 5 6]
 [7 0 4]] 
h =  4
[[8 2 3]
 [0 1 6]
 [7 5 4]] 
h =  5
Visitando # 6
[[8 2 3]
 [1 5 6]
 [0 7 4]] 
h =  5
[[8 2 3]
 [1 5 6]
 [7 4 0]] 
h =  3
Visitando # 7
[[8 2 3]
 [1 5 0]
 [7 4 6]] 
h =  5
Visitando # 8
[[0 2 3]
 [8 1 6]
 [7 5 4]] 
h =  5
[[8 2 3]
 [7 1 6]
 [0 5 4]] 
h =  6
Visitando # 9
[[8 2 3]
 [0 5 6]
 [1 7 4]] 
h =  5
Visitando # 10
[[8 2 0]
 [1 5 3]
 [7 4 6]] 
h =  6
[[8 2 3]
 [1 0 5]
 [7 4 6]] 
h =  6
Visitando # 11
[[2 0 3]
 [8 1 6]
 [7 5 4]] 
h =  6
Visitando 